# AI Creative Writing Assistant — Prototype Demo

This notebook demonstrates the minimal 4-agent prototype pipeline:
1. **Plot Architect** — User prompt → StoryBrief + CharacterRoster + WorldContext
2. **Beat Outliner** — Planning artifacts → StoryOutline
3. **Scene Writer** — Scene outline → SceneDraft (prose)
4. **Style Editor** — SceneDraft → EditFeedback (with revision loop)

In [ ]:
# Setup — ensure the backend package is importable
from ai_writer.pipelines.prototype import build_prototype_pipeline

In [ ]:
# Define the story prompt
PROMPT = (
    "Write a short sci-fi story about a lone engineer on a deep-space relay station "
    "who intercepts a transmission that appears to come from a civilization that went "
    "extinct thousands of years ago. Themes: isolation, discovery, witness. "
    "Contemplative tone with mounting tension. 1 act, 2-3 scenes, ~2000-3000 words."
)

In [ ]:
# Build and run the pipeline
pipeline = build_prototype_pipeline()

result = pipeline.invoke({
    "user_prompt": PROMPT,
    "scene_drafts": [],
    "edit_feedback": [],
    "current_scene_index": 0,
    "revision_count": 0,
    "max_revisions": 2,
    "min_revisions": 1,
    "current_stage": "planning",
    "errors": [],
})

In [ ]:
# Inspect: Story Brief
brief = result["story_brief"]
print(f"Title: {brief.title}")
print(f"Genre: {brief.genre.value}")
print(f"Premise: {brief.premise}")
print(f"Themes: {brief.themes}")
print(f"Tone: formality={brief.tone_profile.formality}, darkness={brief.tone_profile.darkness}")

In [ ]:
# Inspect: Character Roster
roster = result["character_roster"]
for c in roster.characters:
    print(f"\n{c.name} ({c.role.value})")
    print(f"  Motivation: {c.motivation}")
    print(f"  Voice: {c.voice_notes}")
    print(f"  Traits: {c.personality_traits}")

In [ ]:
# Inspect: World Context
world = result["world_context"]
print(f"Period: {world.setting_period}")
print(f"Description: {world.setting_description}")
print(f"\nLocations:")
for loc in world.locations:
    print(f"  - {loc.name}: {loc.description}")
print(f"\nKey Facts:")
for fact in world.key_facts:
    print(f"  - {fact}")

In [ ]:
# Inspect: Story Outline
outline = result["story_outline"]
print(f"Total beats: {outline.total_beats}")
print(f"Total scenes: {outline.total_scenes}")
for act in outline.acts:
    print(f"\nAct {act.act_number}: {act.title}")
    for scene in act.scenes:
        print(f"  Scene {scene.scene_number}: {scene.title}")
        print(f"    Goal: {scene.scene_goal}")
        print(f"    Emotional arc: {scene.emotional_arc}")

In [ ]:
# Inspect: Scene Drafts
drafts = result["scene_drafts"]
for draft in drafts:
    print(f"\n{'='*60}")
    print(f"Scene {draft.act_number}.{draft.scene_number} ({draft.word_count} words)")
    print(f"{'='*60}")
    print(draft.prose[:500] + "..." if len(draft.prose) > 500 else draft.prose)

In [ ]:
# Inspect: Edit Feedback
feedback = result["edit_feedback"]
for fb in feedback:
    status = "APPROVED" if fb.approved else "REVISION NEEDED"
    print(f"\nScene {fb.scene_id}: score={fb.quality_score:.2f} [{status}]")
    print(f"Assessment: {fb.overall_assessment[:200]}")
    if fb.edits:
        print(f"Edits ({len(fb.edits)}):")
        for edit in fb.edits[:3]:
            print(f"  [{edit.severity.value}] {edit.edit_type.value}: {edit.rationale[:80]}")

In [ ]:
# Full Manuscript Output
total_words = sum(d.word_count for d in drafts)
print(f"Total word count: {total_words}")
print(f"Edit rounds: {len(feedback)}")
print("\n" + "="*60)
print("FULL MANUSCRIPT")
print("="*60)
for draft in drafts:
    print(f"\n--- Scene {draft.act_number}.{draft.scene_number} ---\n")
    print(draft.prose)

## Evaluation Notes

After running, consider:
- Did the structured decomposition (schemas) produce better results than a single prompt would?
- Were character voices distinct?
- Did the edit loop meaningfully improve prose quality?
- How many revision rounds occurred per scene?
- Where did the pipeline struggle most?